# 라이브러리 import

In [48]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib import rc # 폰트
from datetime import datetime
import time
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')
# datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')

path = 'C:\\Users\\wldbs\\Desktop\\kart\\'

# 크롤링 데이터 전처리

In [16]:
upper_map= pd.read_csv(path + 'upper_map.csv', encoding = 'utf-8')
lower_map = pd.read_csv(path + 'lower_map.csv', encoding = 'utf-8')

In [22]:
crawl = pd.read_csv('C:\\Users\\wldbs\\Desktop\\kart-game-project\\[data]_crawling\\01_crawling_track.csv', encoding = 'cp949')
request2 = pd.read_csv('C:\\Users\\wldbs\\Desktop\\kart-game-project\\[data]_crawling\\01_crawling_errortrack.csv', encoding = 'cp949')

In [23]:
# merge를 위해 변수명 변경

crawl.columns = ['map_name', '태그', '난이도', '랩', '트랙 길이', '진행 방향', '트랙 분류', 'AI 주행', '첫 등장일',
       '리그 트랙', '라이센스', '약칭']

In [24]:
# 상위 / 하위 맵에 map_name 붙여서 맵 이름으로 출력 가능케 함

merge_upper_crawl = pd.merge(upper_map, crawl, on = 'map_name', how = 'left')
merge_upper_crawl = merge_upper_crawl.drop_duplicates()

merge_lower_crawl = pd.merge(lower_map, crawl, on = 'map_name', how = 'left')
merge_lower_crawl = merge_lower_crawl.drop_duplicates()

# print(merge_upper_crawl.isnull().sum())
# print(merge_lower_crawl.isnull().sum())

In [25]:
# 결측치 대치 (missing 이라는 문자값으로 입력되게 함)
merge_upper_crawl = merge_upper_crawl.fillna('missing')
merge_lower_crawl = merge_lower_crawl.fillna('missing')

In [26]:
# 상위 맵에서 결측치 있는 맵 이름 추출

search_name = merge_upper_crawl.loc[(merge_upper_crawl['약칭'] == 'missing') | (merge_upper_crawl['태그'] == 'missing') | (merge_upper_crawl['난이도'] == 'missing') |(merge_upper_crawl['랩'] == 'missing') |(merge_upper_crawl['트랙 길이'] == 'missing') |(merge_upper_crawl['진행 방향'] == 'missing') |(merge_upper_crawl['트랙 분류'] == 'missing') |(merge_upper_crawl['AI 주행'] == 'missing') |(merge_upper_crawl['첫 등장일'] == 'missing') |(merge_upper_crawl['리그 트랙'] == 'missing') |(merge_upper_crawl['라이센스'] == 'missing')].reset_index()
search_name_df = search_name['map_name'].reset_index()
# search_name_df

In [27]:
name_list = search_name_df.map_name.to_list()

## upper 전처리

In [28]:
# 필요 없는 컬럼 제거
# 제대로 분류가 안 되어있거나, 통계 분석에 필요 없는 내용 삭제

merge_upper_crawl = merge_upper_crawl.drop(['태그', '트랙 분류', '리그 트랙', '라이센스', '약칭', 'AI 주행'], axis = 1)

In [30]:
# 난이도 문자열 - 숫자열로 변경

merge_upper_crawl['난이도'] = merge_upper_crawl.난이도.str.split('[').str[0]
merge_upper_crawl['난이도'] = merge_upper_crawl.난이도.str.split(' ').str[0]
# merge_upper_crawl.난이도.unique()


merge_upper_crawl['난이도'].replace( {'●○○○○' : 1,
                                '●●○○○' : 2,
                                '●●●○○' : 3,
                                '●●●●○' : 4,
                                '●●●●●' : 5,
                                '●●○○○○' : 2  }, inplace = True)

In [31]:
# 랩 문자열 - 숫자열로 변경
#merge_upper_crawl.랩.unique()

merge_upper_crawl['랩'] = merge_upper_crawl.랩.str.split('바').str[0]
# merge_upper_crawl.난이도.unique()

merge_upper_crawl['랩'].replace( {'1' : 1,
                                  '2' : 2,
                                  '3' : 3,
                                  'missing' : 0 }, inplace = True)

In [32]:
# 트랙 길이 문자열 - 숫자열로 변경

merge_upper_crawl['트랙 길이'].unique()

merge_upper_crawl['트랙 길이'] = merge_upper_crawl['트랙 길이'].str.split('k').str[0]
merge_upper_crawl['트랙 길이'] = merge_upper_crawl['트랙 길이'].str.split(' ').str[0]
merge_upper_crawl['트랙 길이'].replace( {'missing' : 0 }, inplace = True)

merge_upper_crawl['트랙 길이'] = merge_upper_crawl['트랙 길이'].astype('float')

In [33]:
# 범주형 변수 따로 변환하지 않기로 함
merge_upper_crawl['진행 방향'].unique()

array(['시계방향', '단선형', 'missing', '반시계방향', '워프형'], dtype=object)

In [34]:
# 날짜형 변환

merge_upper_crawl['첫 등장일'].replace( {'missing' : '0'}, inplace = True)

merge_upper_crawl['년'] = merge_upper_crawl['첫 등장일'].str.split(' ').str[0]
merge_upper_crawl['월'] = merge_upper_crawl['첫 등장일'].str.split(' ').str[1]
merge_upper_crawl['일'] = merge_upper_crawl['첫 등장일'].str.split(' ').str[2]

merge_upper_crawl['년'] = merge_upper_crawl['년'].str.strip('년').str[0:4]
merge_upper_crawl['월'] = merge_upper_crawl['월'].str.split('월').str[0]
merge_upper_crawl['일'] = merge_upper_crawl['일'].str.split('일').str[0]

merge_upper_crawl.fillna('0')

merge_upper_crawl['월'] = merge_upper_crawl['월'].str.rjust(width=2,fillchar='0')        # rlpad
merge_upper_crawl['일'] = merge_upper_crawl['일'].str.rjust(width=2,fillchar='0')        # rlpad

In [35]:
# 출시일 컬럼 생성

merge_upper_crawl['출시일'] = merge_upper_crawl['년'] + merge_upper_crawl['월'] + merge_upper_crawl['일']
merge_upper_crawl

merge_upper_crawl['출시일'] = pd.to_datetime(merge_upper_crawl['출시일'])
merge_upper_crawl

,map_name,cnt_match,log_cnt,난이도,랩,트랙 길이,진행 방향,첫 등장일,년,월,일,출시일
0,빌리지 운하,1438,7.271704,1,3,4.8,시계방향,2004년 6월 1일,2004,06,01,2004-06-01
2,트랙 9,1152,7.050123,2,1,3.5,단선형,2013년 8월 8일,2013,08,08,2013-08-08
3,사막 버려진 오아시스,912,6.816736,missing,0,0.0,missing,0,0,NaN,NaN,NaT
4,코리아 제주 해오름 다운힐,776,6.655440,2,1,5.6,단선형,2021년 8월 12일,2021,08,12,2021-08-12
5,노르테유 허공의 갈림길,756,6.629363,2,3,3.9,반시계방향,2005년 8월 25일,2005,08,25,2005-08-25
6,빌리지 고가의 질주,391,5.971262,2,2,7.4,시계방향,2004년 6월 1일,2004,06,01,2004-06-01
8,비치 해변 드라이브,322,5.777652,2,2,9.9,반시계방향,2013년 7월 11일,2013,07,11,2013-07-11
9,메이플 헤네시스 공원,270,5.602119,missing,0,0.0,missing,0,0,NaN,NaN,NaT
10,메이플 리스항구 언덕길,253,5.537334,missing,0,0.0,missing,0,0,NaN,NaN,NaT
11,빌리지 남산,239,5.480639,2,2,8.2,시계방향,2012년 11월 29일,2012,11,29,2012-11-29


In [36]:
# 첫 등장일 컬럼 삭제

merge_upper_crawl = merge_upper_crawl.drop(['첫 등장일'], axis = 1)

## lower 전처리

In [37]:
# 필요 없는 컬럼 제거
# 제대로 분류가 안 되어있거나, 통계 분석에 필요 없는 내용 삭제

merge_lower_crawl = merge_lower_crawl.drop(['태그', '트랙 분류', '리그 트랙', '라이센스', '약칭', 'AI 주행'], axis = 1)

In [38]:
# 난이도 문자열 - 숫자열로 변경

merge_lower_crawl['난이도'] = merge_lower_crawl.난이도.str.split('[').str[0]
merge_lower_crawl['난이도'] = merge_lower_crawl.난이도.str.split(' ').str[0]
# merge_upper_crawl.난이도.unique()


merge_lower_crawl['난이도'].replace( {'●○○○○' : 1,
                                '●●○○○' : 2,
                                '●●●○○' : 3,
                                '●●●●○' : 4,
                                '●●●●●' : 5,
                                '●●○○○○' : 2  }, inplace = True)

In [39]:
# 랩 문자열 - 숫자열로 변경
#merge_upper_crawl.랩.unique()

merge_lower_crawl['랩'] = merge_lower_crawl.랩.str.split('바').str[0]
# merge_upper_crawl.난이도.unique()

merge_lower_crawl['랩'].replace( {'1' : 1,
                                  '2' : 2,
                                  '3' : 3,
                                  'missing' : 0 }, inplace = True)

In [40]:
# 트랙 길이 문자열 - 숫자열로 변경

merge_lower_crawl['트랙 길이'].unique()

merge_lower_crawl['트랙 길이'] = merge_lower_crawl['트랙 길이'].str.split('k').str[0]
merge_lower_crawl['트랙 길이'] = merge_lower_crawl['트랙 길이'].str.split(' ').str[0]
merge_lower_crawl['트랙 길이'].replace( {'missing' : 0 }, inplace = True)

merge_lower_crawl['트랙 길이'] = merge_lower_crawl['트랙 길이'].astype('float')

In [41]:
# 범주형 변수 따로 변환하지 않기로 함
merge_lower_crawl['진행 방향'].unique()

array(['시계방향', '반시계방향', 'missing', '워프형', '단선형', '8자형'], dtype=object)

In [42]:
# 날짜형 변환

merge_lower_crawl['첫 등장일'].replace( {'missing' : '0'}, inplace = True)

merge_lower_crawl['년'] = merge_lower_crawl['첫 등장일'].str.split(' ').str[0]
merge_lower_crawl['월'] = merge_lower_crawl['첫 등장일'].str.split(' ').str[1]
merge_lower_crawl['일'] = merge_lower_crawl['첫 등장일'].str.split(' ').str[2]

merge_lower_crawl['년'] = merge_lower_crawl['년'].str.strip('년').str[0:4]
merge_lower_crawl['월'] = merge_lower_crawl['월'].str.split('월').str[0]
merge_lower_crawl['일'] = merge_lower_crawl['일'].str.split('일').str[0]

merge_lower_crawl.fillna('0')

merge_lower_crawl['월'] = merge_lower_crawl['월'].str.rjust(width=2,fillchar='0')        # rlpad
merge_lower_crawl['일'] = merge_lower_crawl['일'].str.rjust(width=2,fillchar='0')        # rlpad

In [43]:
# 출시일 컬럼 생성

merge_lower_crawl['출시일'] = merge_lower_crawl['년'] + merge_lower_crawl['월'] + merge_lower_crawl['일']
merge_lower_crawl['출시일'] = pd.to_datetime(merge_lower_crawl['출시일'])

In [44]:
# 첫 등장일 컬럼 삭제

merge_lower_crawl = merge_lower_crawl.drop(['첫 등장일'], axis = 1)

## ~~변경 방법 for문으로 한 번에 해결 시도~~
- for문으로 한 번에 돌릴 수 있을까 생각해서 만들었는데 에러 발생
- 각각 따로 하면 잘 되는데, 왜 안 되는지 확인 필요

In [ ]:
# for df in [merge_upper_crawl, merge_lower_crawl]:
#     # 필요 없는 컬럼 제거
#     # 제대로 분류가 안 되어있거나, 통계 분석에 필요 없는 내용 삭제
#     df = df.drop(['태그', '트랙 분류', '리그 트랙', '라이센스', '약칭', 'AI 주행'], axis = 1)
    
#     # 난이도 문자열 - 숫자열로 변경
#     df['난이도'] = df.난이도.str.split('[').str[0]
#     df['난이도'] = df.난이도.str.split(' ').str[0]

#     df['난이도'].replace( {'●○○○○' : 1,
#                            '●●○○○' : 2,
#                            '●●●○○' : 3,
#                            '●●●●○' : 4,
#                            '●●●●●' : 5,
#                            '●●○○○○' : 2  }, inplace = True)
    
#     # 랩 문자열 - 숫자열로 변경
#     df['랩'] = df.랩.str.split('바').str[0]

#     df['랩'].replace( {'1' : 1,
#                        '2' : 2,
#                        '3' : 3,
#                        'missing' : 0 }, inplace = True)
    
#     # 랩 문자열 - 숫자열로 변경
#     df['랩'] = df.랩.str.split('바').str[0]

#     df['랩'].replace( {'1' : 1,
#                        '2' : 2,
#                        '3' : 3,
#                        'missing' : 0 }, inplace = True)
    
#     # 트랙 길이 문자열 - 숫자열로 변경
#     df['트랙 길이'] = df['트랙 길이'].str.split('k').str[0]
#     df['트랙 길이'] = df['트랙 길이'].str.split(' ').str[0]
#     df['트랙 길이'].replace( {'missing' : 0 }, inplace = True)
#     df['트랙 길이'] = df['트랙 길이'].astype('float')
    
#     # 날짜형 변환    
#     df['첫 등장일'].replace( {'missing' : '0'}, inplace = True)

#     df['년'] = df['첫 등장일'].str.split(' ').str[0]
#     df['월'] = df['첫 등장일'].str.split(' ').str[1]
#     df['일'] = df['첫 등장일'].str.split(' ').str[2]

#     df['년'] = df['년'].str.strip('년').str[0:4]
#     df['월'] = df['월'].str.split('월').str[0]
#     df['일'] = df['일'].str.split('일').str[0]

#     df.fillna('0')

#     df['월'] = df['월'].str.rjust(width=2,fillchar='0')        # rlpad
#     df['일'] = df['일'].str.rjust(width=2,fillchar='0')        # rlpad
    
#     df['출시일'] = df['년'] + df['월'] + df['일']
#     df['출시일'] = pd.to_datetime(df['출시일'])


## 전처리 완료된 upper, lower csv 저장

In [45]:
# 저장하기

merge_upper_crawl.to_csv(path + 'upper_crawl.csv', index=False, encoding = 'utf-8')
merge_lower_crawl.to_csv(path + 'lower_crawl.csv', index=False, encoding = 'utf-8')